# 8_特徴量エンジニアリング
- 前章では「性別」「年齢」「運賃」という3つのデータから生死の予測をたてました
- しかし、「性別」のみで予測した場合との精度の差はみられなかったと思います
- ここで取り組む特徴量エンジニアリングとは、予測モデルに使う特徴量自体を自ら作り出し、精度向上を狙うものです
- 今回は、Cabin（部屋番号）を用いた特徴量エンジニアリングを行います。

## ■写経

### ライブラリインポート（PandasとSciPy）
- 表形式データを簡単に扱うためのPandasをプログラムにインポートします。インポートすることでPandasの機能が有効化されます
- また予測モデルの構築のためにsklearnをインポートします

In [3]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer

### データ読み込み（前章と同じ）
- Pandasの機能を使い、"train.csv"というデータをPythonコードで読み込み、Dataframeオブジェクトとして扱います

In [4]:
train = pd.read_csv("train.csv")

### データを見てみる（前章と同じ）
- head関数をつかって、先頭5行分のデータをダイジェスト表示します

In [5]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### 前準備
- maleを0、femaleを1に置き換えた列を作成します
- Cabin列の頭文字が部屋グレードを表すため、各行のCabinの頭文字を取得し、ダミー変数化します
  - ダミー変数化とは、質的データの各水準をデータカラムとし、2値変数とする操作です。これにより質的データを回帰モデルのような連続値関数で取り扱えるようになります

In [6]:
# Sexを数値に変換(男性:0, 女性:1)
train['Sex_bi'] = train['Sex'].map({'male': 0, 'female': 1})

# Cabinの頭文字を取得
# pd.notnull(x)はxが欠損値でない場合にTrueを返す
# lumbda x: x[0]はxの1文字目を取得
train['Cabin_initial'] = train['Cabin'].apply(lambda x: x[0] if pd.notnull(x) else 'N')
# get_dummiesでone-hotエンコーディング
train = pd.get_dummies(train, columns=['Cabin_initial'])
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,Sex_bi,Cabin_initial_A,Cabin_initial_B,Cabin_initial_C,Cabin_initial_D,Cabin_initial_E,Cabin_initial_F,Cabin_initial_G,Cabin_initial_N,Cabin_initial_T
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,...,0,False,False,False,False,False,False,False,True,False
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,...,1,False,False,True,False,False,False,False,False,False
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,...,1,False,False,False,False,False,False,False,True,False
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,...,1,False,False,True,False,False,False,False,False,False
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,...,0,False,False,False,False,False,False,False,True,False


### lambda関数部分の補足

lambda x: x[0] if pd.notnull(x) else 'N'

このコードでは、lambda関数は各行の'Cabin'列の値xを引数として受け取り、その値が欠損値でない場合（pd.notnull(x)が真）はその値の最初の文字を返し（x[0]）、欠損値の場合は'N'を返すという役割を果たしています。

これを通常のPython関数として書くと次のようになります。

def get_initial(x):
    if pd.notnull(x):
        return x[0]
    else:
        return 'N'

そして、applyメソッドを使う部分をこの関数を使って書き直すと、

train['Cabin_initial'] = train['Cabin'].apply(get_initial)

この関数は一箇所でしか使用していないため、その場で定義してしまうことでコードが簡潔になります。そのような場合にはlambda関数が便利です。

データの流れでみると

元の `Cabin` 列のデータ形式

| Cabin  |
|--------|
| C123   |
| B57B59 |
| D33    |
| NaN    |
| A6     |

そして、Cabinの頭文字を取得した結果

| Cabin_initial |
|---------------|
| C             |
| B             |
| D             |
| N             |
| A             |

### one-hotエンコーディングの補足

次にone-hotエンコーディングについて

1. one-hotベクトルやone-hotエンコーディングの説明:
   one-hotエンコーディングは、カテゴリ変数を表現する一つの方法であり、特に機械学習モデルで使用されます。カテゴリ変数の各値を、その値の位置のみが1で他のすべてが0のベクトルで表現します。例えば、カテゴリ変数が「赤」「緑」「青」の3つの値を持つ場合、これをone-hotエンコーディングすると以下のようになります。

    | 元の値 | 赤 | 緑 | 青 |
    |-------|---|---|---|
    | 赤    | 1 | 0 | 0 |
    | 緑    | 0 | 1 | 0 |
    | 青    | 0 | 0 | 1 |

2. なぜ使う必要があるのか:
   機械学習モデルは通常、数値データを入力として扱います。カテゴリデータ（例：色、都市名、商品カテゴリ等）は何らかの方法で数値に変換することで、機械学習（Scikit-learn等）で使いやすくなります。
   しかし、単純にカテゴリを数値にマッピングすると（例：「赤」=1、「緑」=2、「青」=3）、これは「赤」<「緑」<「青」のような順序関係を暗示してしまう可能性がありone-hotエンコーディングを用いることで、各カテゴリが等しく独立した特性であると捉えることができます。（ダミー変数化ともいう

3. テーブルデータを使って、今回のcabin列のデータがどのように変化するのか:

元の `Cabin` 列のデータは以下のような形式で格納

| Cabin  |
|--------|
| C123   |
| B57B59 |
| D33    |
| NaN    |
| A6     |

そして、Cabinの頭文字を取得した結果が以下のようになります：

| Cabin_initial |
|---------------|
| C             |
| B             |
| D             |
| N             |
| A             |

ここで 'N' は `Cabin` 列の欠損値を表しています。この `Cabin_initial` 列をone-hotエンコーディングした結果が先程示した表です。

   今回のケースでは、`Cabin_initial`列の値は 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'T', 'N' の9つ（'N'は欠損値）となります。これをone-hotエンコーディングすると以下のようになります。

   元のテーブル:

   | Cabin_initial |
   |---------------|
   | C             |
   | B             |
   | D             |
   | N             |
   | A             |


one-hotエンコーディング後:

| Cabin_initial_A | Cabin_initial_B | Cabin_initial_C | Cabin_initial_D | Cabin_initial_N | ... |
|-----------------|-----------------|-----------------|-----------------|-----------------|-----|
| 0               | 0               | 1               | 0               | 0               | ... |
| 0               | 1               | 0               | 0               | 0               | ... |
| 0               | 0               | 0               | 1               | 0               | ... |
| 0               | 0               | 0               | 0               | 1               | ... |
| 1               | 0               | 0               | 0               | 0               | ... |

上記の表は、Cabinの頭文字がそれぞれ 'C', 'B', 'D', 'N', 'A' である各行をone-hotエンコーディングした結果を示しています。例えば最初の行では、元の値が 'C' だったので、新しい 'Cabin_initial_C' 列が1で、他のすべての列が0となっています。

このようにして、`pd.get_dummies(train, columns=['Cabin_initial'])` の結果、各 'Cabin_initial' の値に対応する新しい列が追加され、それぞれの行がその元の 'Cabin_initial' の値を反映したone-hotベクトルに変換されます。

### 重回帰モデルで使用する特徴量と目的変数を定義し、欠損値処理をする
- 今回は性別、年齢、客室ランク（特徴量）が生死（目的変数）と関係するという仮説をたてています

In [7]:
# 欠損値保管は最頻値とする
imputer = SimpleImputer(strategy='most_frequent')

# 特徴量の選択。Sex_bi列とCabinのダミー変数
features = ['Sex_bi'] + [col for col in train.columns if 'Cabin_initial' in col]
X_train = train[features]
y_train = train['Survived']

# Fill missing values
X_train = imputer.fit_transform(X_train)

### 重回帰モデルの作成と学習

In [8]:
# 空の予測モデルを作成
model = LinearRegression()
# 学習データを与えて学習
model.fit(X_train, y_train)

LinearRegression()

### テストデータに対する予測の実施
- 上記で学習したモデルに、テスト用のデータを予測させます

In [9]:
# 提出用テストデータを読み込みます（すなわち、学習したモデルの精度を測るための検証データセットです）
test = pd.read_csv('test.csv')

# 提出用データも同様に、'Sex'列を0、1に変換します
test['Sex_bi'] = test['Sex'].map({'male': 0, 'female': 1})

test['Sex_bi'] = test['Sex'].map({'male': 0, 'female': 1})
test['Cabin_initial'] = test['Cabin'].apply(lambda x: x[0] if pd.notnull(x) else 'N')
test = pd.get_dummies(test, columns=['Cabin_initial'])

# 提出用データの特徴量を分割します
for col in features:
    if col not in test.columns:
        test[col] = 0  # add missing feature in test data
X_test = test[features]
X_test = imputer.transform(X_test)

# 提出用データに対する予測を行います
predictions = model.predict(X_test)

# predictionsの各データを四捨五入してpredictionsに再代入する
predictions = [round(pred) for pred in predictions]

predictions

[0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,


### 提出用データの作成
- 提出データは、passenger id, Survivedという2列のデータ形式とします
- データをcsv形式で出力します

In [11]:
# 提出用のDataFrameを作成します
submission = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': predictions})

# 提出用のcsvファイルを作成します
submission.to_csv('submission3.csv', index=False)

### 答え合わせ
- この問題は、世界的なAIコンペサイトであるKaggleの練習問題コンペになっています
- そこでKaggleに上記の結果を提出して、スコア（予測精度）を確認しましょう
- https://www.kaggle.com/competitions/titanic/overview